# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [59]:
import os
import io
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = pd.concat([data, dataFrameFromDirectory("emails/spam", "spam")]);
data = pd.concat([data, dataFrameFromDirectory("emails/ham", "ham")])

#For Pandas 1.3:
#data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
#data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [60]:
data.head()

,message,class
emails/spam/00223.349b9b0748ee72bad60729ffaae2cc00,This is a multi-part message in MIME format.\n...,spam
emails/spam/00276.a6e447390e371ddba7cee092bb0ec98f,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",spam
emails/spam/00427.fa1252c91a3b89bb64bc2bc217725e26,"Hi,\n\n\n\nI'm a college dropout. I work abou...",spam
emails/spam/00473.7086ec944a1245a002b04547a433c887,"<TABLE id=AutoNumber2 style=""BORDER-COLLAPSE: ...",spam
emails/spam/00438.41295e1df4b651b7611316331b8468e4,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",spam


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [61]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

print(counts)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

  (0, 53337)	7
  (0, 31369)	3
  (0, 38139)	1
  (0, 41863)	1
  (0, 36932)	3
  (0, 30375)	1
  (0, 37233)	1
  (0, 24883)	1
  (0, 8518)	3
  (0, 12397)	3
  (0, 17466)	6
  (0, 54656)	17
  (0, 53112)	9
  (0, 43154)	1
  (0, 15912)	3
  (0, 58662)	1
  (0, 1193)	1
  (0, 54110)	2
  (0, 22353)	2
  (0, 6365)	1
  (0, 41217)	3
  (0, 40807)	3
  (0, 24806)	5
  (0, 43875)	3
  (0, 44148)	3
  :	:
  (2999, 27733)	1
  (2999, 46836)	1
  (2999, 52648)	3
  (2999, 28859)	1
  (2999, 50659)	2
  (2999, 49454)	1
  (2999, 50925)	1
  (2999, 41024)	1
  (2999, 53349)	1
  (2999, 35283)	1
  (2999, 46689)	1
  (2999, 17660)	1
  (2999, 46106)	2
  (2999, 31860)	2
  (2999, 59754)	1
  (2999, 14044)	1
  (2999, 37987)	1
  (2999, 10400)	2
  (2999, 44135)	2
  (2999, 38050)	1
  (2999, 25492)	1
  (2999, 18072)	1
  (2999, 50000)	1
  (2999, 56914)	1
  (2999, 28579)	1


MultinomialNB()

Let's try it out:

In [68]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [69]:
examples2 = ['get free Viagra now!!!', 'Hi Bob, how about a game of golf tomorrow?']
example_counts2 = vectorizer.transform(examples2)
predictions2 = classifier.predict(example_counts2)
predictions2

array(['spam', 'ham'], dtype='<U4')

In [64]:
from sklearn.model_selection import train_test_split
import numpy as np

train, test = train_test_split(data, test_size=0.2)

vectorizer2 = CountVectorizer()
classifier2 = MultinomialNB()

In [65]:
counts_train = vectorizer2.fit_transform(train['message'].values)

In [66]:
targets_train = train['class'].values
classifier2.fit(counts_train, targets_train)

MultinomialNB()

In [79]:
examples3 = test['message'].values

example_counts3 = vectorizer2.transform(examples3)

predictions3 = classifier2.predict(example_counts3)
predictions3

array(['ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham',
       'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham',
       'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham',
       'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam',
       'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
       'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham',
  